# Factorizing the synthetic dataset

In [ ]:
import numpy as np
import pandas as pd

import networkx as nx

%matplotlib inline
import matplotlib.pyplot as plt

Let's import `TRMFREgressor` which provides scikit-like, but still
incomplatible interface.

In [ ]:
from trmf import TRMFRegressor

<br>

## Synthetic dataset

Let's fix the seed

In [ ]:
random_state = np.random.RandomState(8945634)

We will play with a latent autoregression dataset that we are about to generate.

In [ ]:
n_samples, n_components, n_targets, n_order = 120, 4, 16, 8

<br>

### Generating stationary lag polynomials

Generate some roots within the unit cicrle:
* if $r \sim \mathrm{U}[0, 1]$ and $\phi \sim \mathrm{U}[0, 2\pi]$ then
the complex-values random variable $Z = \sqrt{r} e^{i \phi}$ has a uniform
distribution within the unit disc.

In [ ]:
rad = random_state.uniform(0, 1, size=(n_components, n_order))
phi = random_state.uniform(0, 2, size=(n_components, n_order)) * np.pi

phi /= 4

roots = np.sqrt(rad) * (np.cos(phi) + np.sin(phi) * 1.j)

#### Proof

Indeed, in multivariate calculus the change of variables under integration is performed thus
$$
\int_R f(x) dx
    = \int_S
        f(g(u)) \lvert J \rvert du
    \,, $$
where the Jacobian is given by $\tfrac{\partial g}{\partial u^{\mathrm{T}}}$. Therefore, for
any measurable rectangle $A\times B$ in $\mathbb{R}^2$ we have
$$
\int_0^1 \int_0^{2\pi} \tfrac1{2 \pi}
    1_{A \times B} \bigl(\sqrt{r} \cos \phi, \sqrt{r} \sin \phi\bigr) dr d\phi
    = \bigl[
        x,y = \sqrt{r} \cos \phi, \sqrt{r} \sin \phi,\, \det J = \tfrac12
    \bigr]
    % = \iint_{B[0,1]} \tfrac1{2 \pi}
    %     1_{A \times B} \bigl(x, y\bigr) 2 \tfrac12 dx dy
    = \iint_{B[0,1]} \tfrac1{\pi} 1_{A \times B} \bigl(x, y\bigr) dx dy
    \,, $$
where $B[0, 1] = \{x\in \mathbb{R}^2\colon \|x\|_2 \leq 1\}$ and the Jacobian is 
$$
\det J
    = \begin{vmatrix}
        \tfrac{\partial x}{\partial r} & \tfrac{\partial x}{\partial \phi} \\
        \tfrac{\partial y}{\partial r} & \tfrac{\partial y}{\partial \phi} \\
    \end{vmatrix}
    = \begin{vmatrix}
        \tfrac1{2\sqrt{r}} \cos \phi & - \sqrt{r} \sin \phi \\
        \tfrac1{2\sqrt{r}} \sin \phi &   \sqrt{r} \cos \phi
    \end{vmatrix}
    = \tfrac1{2\sqrt{r}} \sqrt{r} \cos^2 \phi - \tfrac1{2\sqrt{r}} (-\sqrt{r}) \sin^2 \phi
    = \tfrac12 \cos^2 \phi + \tfrac12 \sin^2 \phi
    \,. $$
Note that the area of $B[0, 1]$ is $r^2 \pi = 1\cdot \pi$.

Next to avoid negative autocorrelation, let's confine the roots to the right-half of the unit disc.

In [ ]:
# roots.real = np.abs(roots.real)

Let's make sure that the complex roots are in conjugate pairs, so that the lag ploynomial has real coefficients.

* we keep at least two real roots and fill the rest with conjugate pairs.

In [ ]:
n_cplx = max(0, n_order - 1) // 2

n_real = n_order - n_cplx * 2

roots = np.concatenate([
    roots[:, :n_cplx], np.conj(roots[:, :n_cplx]), np.real(roots[:, -n_real:])
], axis=-1)

For the lag polynomial we have the following:
$$
    \Phi(L)
        = 1 - \sum_{k=1}^p \phi_k L^k
        = L^p \Bigl(
            L^{-p} - \sum_{k=1}^p \phi_k L^{k-p}
        \Bigr)
        = L^p \Bigl(
            z^p + \sum_{k=1}^p (-\phi_k) z^{p-k}
        \Bigr) \Big\vert_{z = L^{-1}}
        = L^p \prod_{k=1}^p (z - z_k) \Big\vert_{z = L^{-1}}
        = \prod_{k=1}^p (1 - z_k L)
    \,, $$

where $(zk)_{k=1}^p\in \mathbb{C}$ are the roots of the $p$-th order polynomial
$q(z) = z^p - \phi_1 z^{p-1} - \cdots - \phi_p = \phi_0 z^p + \sum_{k=1}^p (-\phi_k) z^{p-k}$
with $\phi_0 = 1$.

#### Some intuition

We have the following formal series in terms of the lag operator $L$, provided $\lvert z \rvert < 1$, $z\in \mathbb{C}$:
$$
    (1 - z L)^{-1} = \sum_{k\geq 0} z^k L^k
    \,. $$
If each $z_k$ is such, then its associated order-$1$ lag polynomial $(1-z_k L)$ is invertible. Since the lag operators
commute, the whole lag polynomial $\Phi(L)$ is invertible.

Therefore, if all $z_k\in \mathbb{C}$ lie inside the unit circle, then the lag polynomial represents a stationary (causal) autoregression process of order $p$.

#### From the roots to the ploynomial

The function `np.poly` computes the coefficients of a polynomial $q(z)$ which has roots $(z_k)_{k=1}^p\in \mathbb{C}$,
where
$$
q(z)
    = \alpha_0 z^p + \sum_{k=1}^p \alpha_k z^{p - k}
    = \prod_{k=1}^p (z - z_k)
    \,, $$
with $\alpha_0=1$. Therefore to the get the coefficients of the corresponding lag
polynomial we just need to map $(z_k)_{k=1}^p \mapsto (\alpha_k)_{k=0}^p$ and then
flip the sign of each one $\phi_k = - \alpha_k$ for $k=1,\,\ldots,\,p$.

In [ ]:
real_phi = np.stack([- np.poly(zeroes)[1:] for zeroes in roots], axis=0)

Let's make sure the roots are within the unit circle.

In [ ]:
colors = np.array([[c] * n_order for c in ["C0", "C1", "C2", "C3"]]).ravel()

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection="polar")
ax.scatter(np.angle(roots), np.abs(roots), c=colors, s=50)  #, c=colors, s=area, cmap='hsv', alpha=0.75)
ax.set_rlim(0, 1.1)
plt.show()

Now $y_{t-p:t}$ returns $y_{t-p}, y_{t-p+1},\,\ldots,\,y_{t-1}$, which means that
to get the next $y_t$ we must multiply each by $\phi_p,\,\ldots,\,\phi_1$ respectively,
i.e. $\phi$ in reverse order.

In [ ]:
real_ar_coef = real_phi[:, ::-1].copy()

plt.imshow(real_ar_coef, cmap=plt.cm.hot)

Simulate the autoregressive process.

In [ ]:
noise = random_state.normal(scale=0.1, size=(n_samples, n_components))
for t in range(n_order, n_samples):
    # the columns in `real_ar_coef` are in ordered from $p$, the least
    #  recent lag, up to $1$, the most recent lag.
    noise[t] += np.einsum("il,li->i", real_ar_coef, noise[t-n_order:t])
#     noise[t] = np.einsum("il,li->i", real_ar_coef, noise[t-n_order:t])

real_factors = noise.copy()

Generate factor loadings

In [ ]:
real_loadings = random_state.uniform(-1, 2, size=(n_components, n_targets))
real_loadings = np.maximum(real_loadings, 0)

noise = random_state.normal(scale=0.1, size=(n_samples, n_targets))

mean = random_state.normal(50, scale=0, size=(1, n_targets))
targets = np.dot(real_factors, real_loadings) + noise + mean

Plot the factors

In [ ]:
n_cols = 3
n_rows = (n_components + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 1.5*n_rows),
                         sharex=True, sharey=False)
for j, ax in zip(range(n_components), axes.ravel()):
    ax.plot(real_factors[:, j])

plt.show()
plt.close()

Plot the observed series

In [ ]:
n_cols = 3
n_rows = (n_targets + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 1.5*n_rows), sharex=True, sharey=False)
for j, ax in zip(range(n_targets), axes.ravel()):
    ax.plot(targets[:, j], lw=2)
    ax.set_title(f"""target {j}""")

plt.tight_layout()
plt.show()

In [ ]:
plt.imshow(real_loadings, cmap=plt.cm.hot)

In [ ]:
plt.plot(np.linalg.norm(real_loadings, ord=1, axis=0))

<br>

Split the synthetic data into train and test periods

In [ ]:
from sklearn.model_selection import train_test_split

train_targets, test_targets = train_test_split(
    targets, test_size=.25, shuffle=False)

Never forget to centre and scale the train dataset!

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

scl = StandardScaler(with_mean=True, with_std=True)

Let's fit a model with more factors but the same AR$(p)$ latent process.

In [ ]:
n_order, n_components = 8, 4

Original setup: no intercept and no regressors

In [ ]:
C_Z, C_F, C_phi = 5e-2, 5e-1, 1e-6
eta_Z, eta_F, adj = 0.95, 0.0, None
C_B, regressors, fit_intercept = 0., None, True

n_max_mf_iter = 2

Better results: fir intercept, place more emphasis on latent autoregression,
regularize the autoregression coefficients more.

In [ ]:
C_Z, C_F, C_phi = 5e-2, 5e-1, 1e-2
eta_Z, eta_F, adj = 0.99, 0.0, None
C_B, regressors, fit_intercept = 0., None, True
n_max_mf_iter = 5  # default

RUN!

In [ ]:
trmf = TRMFRegressor(n_components, n_order, C_Z=C_Z, C_F=C_F, C_phi=C_phi,
                     eta_Z=eta_Z, eta_F=eta_F, adj=adj, C_B=C_B,
                     fit_regression=False, fit_intercept=fit_intercept,
                     nonnegative_factors=True, n_max_mf_iter=n_max_mf_iter)

YY = scl.fit_transform(train_targets)
trmf.fit(YY)

The estimated coefficients

In [ ]:
plt.imshow(trmf.loadings_, cmap=plt.cm.hot)

In [ ]:
plt.imshow(trmf.ar_coef_, cmap=plt.cm.hot)

In [ ]:
plt.plot(trmf.intercept_.T)

In [ ]:
plt.imshow(trmf.coef_, cmap=plt.cm.hot)

Let's have a look at the actual values

In [ ]:
trmf.loadings_

In [ ]:
trmf.ar_coef_

In [ ]:
trmf.intercept_

In [ ]:
trmf.coef_

Set the forecast horizon

In [ ]:
n_ahead, n_horizon = len(test_targets), 24

Forecast the latent factors and paste them with the ones inferred from the train.

In [ ]:
forecast_factors = trmf.forecast_factors(n_ahead + n_horizon)

factors = np.concatenate([trmf.factors_, forecast_factors], axis=0)

Compute train estimates

In [ ]:
estimated = np.dot(trmf.factors_, trmf.loadings_)
estimated += trmf.intercept_

if regressors is not None:
    estimated += np.dot(regressors, trmf.coef_)

Determine which regressors to use

In [ ]:
predicted_features = None
if regressors is not None:
    predicted_features = np.concatenate([
        test_features, np.zeros((n_horizon, test_features.shape[1])),
    ], axis=0)

Compute the predictions:
$$ 
    \hat{Y}_{t+h\mid t}
        = \hat{X}_{t+h\mid t} F
    \,,\quad
    \hat{X}_{t+h\mid t}
        = \sum_{k=1}^p \mathop{\text{diag}}\bigl(\hat{\theta}_{\cdot k}\bigr) \hat{X}_{t+h-k\mid t}
    \,. $$

In [ ]:
predicted = trmf.predict(X=predicted_features, n_ahead=n_ahead + n_horizon)

# paste the train targets and the dynamic forecast
predicted = np.concatenate([estimated, predicted], axis=0)

predicted_target = scl.inverse_transform(predicted)

<br>

### Analysis

Plot the dynamics of the latent factors

In [ ]:
n_cols = 3
n_rows = (n_components + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 1.5*n_rows),
                         sharex=True, sharey=False)

for j, ax in zip(range(n_components), axes.flat):
    ax.plot(factors[:-(n_ahead + n_horizon), j], lw=2)
    ax.plot(factors[:-n_horizon, j], zorder=-1)
    ax.plot(factors[:, j], zorder=-2, alpha=0.5)

for ax in axes.flat[n_components:]:
    y_min, y_max = ax.get_ylim()
    x_min, x_max = ax.get_xlim()
    ax.plot([0, len(factors)], [y_min, y_max], c="k", lw=2, alpha=.25)
    ax.plot([0, len(factors)], [y_max, y_min], c="k", lw=2, alpha=.25)

plt.show()
plt.close()

Metrics

In [ ]:
from sklearn.metrics import mean_squared_error

train_predicted_target = predicted_target[:-n_ahead-n_horizon:]

trmf_mse = mean_squared_error(train_targets, train_predicted_target)
lastknown_mse = mean_squared_error(train_targets[1:], train_targets[:-1])


print(f"""train >>>\nTRMF: {trmf_mse}\nRunning Last: {lastknown_mse}""")

In [ ]:
from sklearn.metrics import mean_squared_error

if n_horizon > 0:
    test_predicted_target = predicted_target[-n_ahead-n_horizon:-n_horizon]
else:
    test_predicted_target = predicted_target[-n_ahead:]

trmf_mse = mean_squared_error(test_targets, test_predicted_target)
last_mse = mean_squared_error(test_targets, train_targets[[-1] * len(test_targets)])
lastknown_mse = mean_squared_error(test_targets[1:], test_targets[:-1])


print(f"""test >>>\nTRMF: {trmf_mse}\nLast train: {last_mse}\nRunning Last: {lastknown_mse}""")

In [ ]:
n_cols = 4
n_rows = (n_targets + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 1.5 * n_rows),
                         sharex=True, sharey=False)

for j, ax in zip(range(n_targets), axes.flat):
    ax.plot(targets[:, j], lw=2)
    ax.plot(predicted_target[:, j], zorder=2)
    ax.axvspan(0, len(train_targets) - 1, color="k", zorder=-1, alpha=0.05)
    ax.set_title(f"""target {j}""")

for ax in axes.flat[n_targets:]:
    y_min, y_max = ax.get_ylim()
    x_min, x_max = ax.get_xlim()
    ax.plot([0, len(predicted_target)], [y_min, y_max], c="k", lw=2, alpha=.25)
    ax.plot([0, len(predicted_target)], [y_max, y_min], c="k", lw=2, alpha=.25)
#     ax.set_frame_on(False) ; ax.set_xticks([]) ; ax.set_yticks([])

plt.tight_layout()
plt.show()

In fact the model will be reestimated upon arrival of new data, so
this validation strategy, where we compare dynamic forecasts with
the actual data is incompatible with the usage scenario.

In [ ]:
plt.plot(trmf.factors_)

In [ ]:
X = np.empty((YY.shape[0], 0)) if regressors is None else regressors
fitted = np.dot(trmf.factors_, trmf.loadings_) + np.dot(X, trmf.coef_) + trmf.intercept_

plt.plot(np.abs(fitted - YY).std(axis=0))

In [ ]:
assert False, """STOP!"""

<br>

#### Extra

Generate some basic features

In [ ]:
features = np.zeros_like(targets[:, :2])
features[2:] = targets[:-2, :2]

In [ ]:
G = nx.erdos_renyi_graph(n_targets, 0.2, directed=True)

In [ ]:
adj = nx.adjacency_matrix(G)
adj = None

<br>